In [1]:
import os
import re
import csv
import sys
import ast 
import json
import time
import urllib
import random
import spotipy
import requests
import datetime
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
client_id = '1a23ed1ca359485ab81d524206dba0f6'
client_secret = 'af0e14fdb19a49dfb19ef0c1ea1d42ea'
username = 'qnwv65t11cplaz4dikhl4mjgi'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

Extracting Spotify data on your favourite artist via Python<br>
https://medium.com/@RareLoot/extracting-spotify-data-on-your-favourite-artist-via-python-d58bc92a4330<br>
https://github.com/plamere/spotipy/blob/master/examples/artist_albums.py

In [29]:
# 取得歌手資訊
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    data = items[0]
    if len(items) > 0:
        artist_name = items[0]['name']
        artist_id = items[0]['id']
        artist_url = items[0]['external_urls']['spotify']
        artist_info = (artist_name,artist_id,artist_url)
        #print(artist_info)
        filename = 'csv/albums/df_albums_' + artist_info[0] + '.csv'
        return items[0], filename
    else:
        return None

# 取得歌手的所有專輯
def show_artist_albums(artist_info, filename):
    albums = []
    # album_type有四種：album、single、appears_on、compilation (include_groups=album,single)
    results = sp.artist_albums(artist_info['id'], album_type='album')
    albums.extend(results['items'])
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])
    # 移除重複的album
    seen = set()
    # 按照專輯的明成排序
    albums.sort(key=lambda album: album['name'].lower())
    list_album_name = []
    list_album_id = []
    list_album_uri = []
    list_album_href = []
    for album in albums:
        album_name = album['name']
        album_id = album['id']
        album_uri = album['uri']
        album_href = 'https://open.spotify.com/album/' + album_id
        if album_name not in seen:
            #print(album_name)
            seen.add(album_name)
            list_album_name.append(album_name)
            list_album_id.append(album_id)
            list_album_uri.append(album_uri)
            list_album_href.append(album_href)
    #print(list_album_name_new)
    #print(list_album_id_new)
    #print(list_album_uri_new)
    df_albums = pd.DataFrame(list(zip(list_album_name,list_album_id,list_album_uri,list_album_href)), columns =['Name','Id','Uri','Href'])
    df_albums.drop_duplicates(subset='Name', keep='first', inplace=True)
    df_albums.reset_index(drop=True, inplace=True)
    #df_albums.to_csv(filename,header=0,index=0)
    return df_albums

In [4]:
artist_info, filename = get_artist('Taylor Swift')
if artist_info:
    df_albums = show_artist_albums(artist_info, filename)
else:
    print("Can't find that artist")
    pass
df_albums

,Name,Id,Uri,Href
0,1989,4uUAUqIfSomFTbbjGp3TYp,spotify:album:4uUAUqIfSomFTbbjGp3TYp,https://open.spotify.com/album/4uUAUqIfSomFTbb...
1,1989 (Big Machine Radio Release Special),6EsTJnpahwW6xX20zvqQgZ,spotify:album:6EsTJnpahwW6xX20zvqQgZ,https://open.spotify.com/album/6EsTJnpahwW6xX2...
2,1989 (Deluxe Edition),34OkZVpuzBa9y40DCy0LPR,spotify:album:34OkZVpuzBa9y40DCy0LPR,https://open.spotify.com/album/34OkZVpuzBa9y40...
3,1989 (Deluxe),5LVuISYu2g4YbyxYhFPIbK,spotify:album:5LVuISYu2g4YbyxYhFPIbK,https://open.spotify.com/album/5LVuISYu2g4Ybyx...
4,Fearless,3Mvk2LKxfhc2KVSnDYC40I,spotify:album:3Mvk2LKxfhc2KVSnDYC40I,https://open.spotify.com/album/3Mvk2LKxfhc2KVS...
5,Fearless (Big Machine Radio Release Special),3EzFY9Rg0PpbADMth746zi,spotify:album:3EzFY9Rg0PpbADMth746zi,https://open.spotify.com/album/3EzFY9Rg0PpbADM...
6,Fearless (International Version),08CWGiv27MVQhYpuTtvx83,spotify:album:08CWGiv27MVQhYpuTtvx83,https://open.spotify.com/album/08CWGiv27MVQhYp...
7,Fearless (Karaoke Version),1CYlmaXajTC59VJWSSeE7Y,spotify:album:1CYlmaXajTC59VJWSSeE7Y,https://open.spotify.com/album/1CYlmaXajTC59VJ...
8,Fearless (Platinum Edition),2gP2LMVcIFgVczSJqn340t,spotify:album:2gP2LMVcIFgVczSJqn340t,https://open.spotify.com/album/2gP2LMVcIFgVczS...
9,Fearless Karaoke,6vRfYCQ1mKKfnB6D7R4N5p,spotify:album:6vRfYCQ1mKKfnB6D7R4N5p,https://open.spotify.com/album/6vRfYCQ1mKKfnB6...


In [6]:
# 匯入csv檔案
def import_csv(filepath):
    df = pd.read_csv(filepath, delimiter = ',')
    return df

In [122]:
# 輸出歌手名稱+Id的csv
with open("csv/df_artists(with ID).csv", "a", encoding="utf_8_sig", newline='') as csvfile:
    writer = csv.writer(csvfile)
    df_artists = pd.read_csv('csv/df_artists.csv')
    base = 7700
    df = df_artists[['artist_name']][base:base+100]
    list_artist_name = []
    list_artist_id = []
    #第一次寫入
    #writer.writerow(['Name', 'Id'])
    for i in range(len(df)):
        artist_name = df['artist_name'][base+i]
        try:
            artist_info, filename = get_artist(artist_name)
            artist_id = artist_info['id']
            #print(artist_id)
            list_artist_name.append(artist_name)
            list_artist_id.append(artist_id)
        except:
            list_artist_id.append('')
            pass
        #之後寫入
        writer.writerow([artist_name, artist_id])

In [16]:
# 新增DateFrame歌手連結的欄位
df2 = import_csv('csv/df_artists(with ID).csv')
list_astist_url = []
for i in range(len(df2)):
    Id = df2['Id'][i]
    Url = 'https://open.spotify.com/artist/' + Id
    list_astist_url.append(Url)
df2['Url'] = list_astist_url
df2.to_csv('csv/df_artists(with ID+Url).csv',index=0,encoding="utf_8_sig")
df2

,Name,Id,Url
0,Alan Walker,7vk5e3vY1uw9plTHJAMwjN,https://open.spotify.com/artist/7vk5e3vY1uw9pl...
1,Avicii,1vCWHaC5f2uS3yhpwWbIA6,https://open.spotify.com/artist/1vCWHaC5f2uS3y...
2,Alec Benjamin,5IH6FPUwQTxPSXurCrcIov,https://open.spotify.com/artist/5IH6FPUwQTxPSX...
3,Aaron Smith,77qukKzmdI5BeMBR7WGUrN,https://open.spotify.com/artist/77qukKzmdI5BeM...
4,Austin Mahone,04abdnqPQe2N4fjztDea6z,https://open.spotify.com/artist/04abdnqPQe2N4f...
...,...,...,...
7704,Zag Van Rigg,0Z46T8Mrh9GLffk6wDpjfS,https://open.spotify.com/artist/0Z46T8Mrh9GLff...
7705,Zennus,0P3mPmgN2mnVEZ2RSHuue4,https://open.spotify.com/artist/0P3mPmgN2mnVEZ...
7706,Zagreb Philharmonic Chorus,1yfG47oyRrO6Aj31HDP5kk,https://open.spotify.com/artist/1yfG47oyRrO6Aj...
7707,Zematary,4T50fumUWw6VefE6iwl0jP,https://open.spotify.com/artist/4T50fumUWw6Vef...


In [32]:
# 輸出歌手名稱+專輯名稱+專輯Id的csv
df_artists = pd.read_csv('csv/df_artists.csv')
for num in range(0,10):
    name = df_artists['artist_name'][num]
    print(name)
    artist_info, filename = get_artist(name)
    empty_album = []

    # 歌手存在
    if artist_info:
        df_albums = show_artist_albums(artist_info, filename)
        # 歌手沒有專輯data
        if df_albums.empty == True:
            empty_album.append(name) #沒有專輯data的歌手名稱放入empty_album列表中
        # 歌手有專輯data
        if df_albums.empty == False:
            df_albums.to_csv('csv/albums/df_albums_'+ name +'.csv',index=0,encoding="utf_8_sig")
    # 歌手不存在
    else:
        print("Can't find that artist")
        pass
#df_albums

Alan Walker
Avicii
Alec Benjamin
Aaron Smith
Austin Mahone
Afrojack
AnDyWuMUSICLAND


IndexError: list index out of range